In [1]:
import csv
import os
import re
from collections import defaultdict
from datetime import datetime
from datetime import timedelta

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Préstamos

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 23.1 s, sys: 1.01 s, total: 24.1 s
Wall time: 25.6 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Country codes

## LOC/MARC

http://www.loc.gov/marc/countries/countries_code.html

In [20]:
loc_marc_df = pd.read_csv('codes-loc-marc-countries.csv', sep='\t', names=['code','country'])
loc_marc_sr = pd.Series(data=list(loc_marc_df['country']), index=list(loc_marc_df['code']))

In [21]:
loc_marc_sr.head()

aa                          Albania
abc                         Alberta
-ac     Ashmore and Cartier Islands
aca    Australian Capital Territory
ae                          Algeria
dtype: object

## ISO

http://www.nationsonline.org/oneworld/country_code_list.htm

In [9]:
iso_df = pd.read_csv('codes-iso-countries.csv', sep='\t', names=['country', 'iso-2', 'iso-3', 'code'])
iso2_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-2']))
iso3_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-3']))

In [10]:
iso2_sr.head()

af       Afghanistan
ax     Aland Islands
al           Albania
dz           Algeria
as    American Samoa
dtype: object

In [11]:
iso3_sr.head()

afg       Afghanistan
ala     Aland Islands
alb           Albania
dza           Algeria
asm    American Samoa
dtype: object

## Mine

In [64]:
missing_codes_dict = {
    'uk': 'United Kingdom',
    'en': 'England',
    'spa': 'Spain',
    'CK': 'Colombia',
    'eng': 'England',
    'ger': 'Germany',
    'uy|': 'Uruguay',
    'SP': 'Spain',
    'AG': 'Argentina',
    'por': 'Portugal',
    'jap': 'Japan',
    'ur': 'Uruguay',
    'fl': 'Florida',
    'flo': 'Florida',
    'fre': 'French Guiana',
    'he': 'Heard Island and Mcdonald Islands',
    'ka': 'Kansas',
    'lou': 'Louisiana',
    'pu': 'Puerto Rico',
    'wa': 'Washington (State)',
}

# Autores

In [18]:
autores_df = pd.DataFrame()

## Nombre

In [14]:
prestamos_df['AUTOR'].value_counts()

                                                                409373
García Márquez, Gabriel, 1927-2014                               13207
Colombia.                                                        11526
Hergé, 1907-1983.                                                11510
Browne, Anthony, 1946-.                                           8653
Foucault, Michel, 1926-1984.                                      6849
Cortázar, Julio, 1914-1984.                                       6478
Nietzsche, Friedrich Wilhelm, 1844-1900.                          5876
Da Coll, Ivar, 1962-                                              5350
Kasza, Keiko.                                                     5336
Shakespeare, William, 1564-1616.                                  5012
Verne, Jules, 1828-1905.                                          4394
Instituto Colombiano de Normas Técnicas y Certificación.          4318
Dostoievski, Fedor Mijailovich, 1821-1881.                        4296
Rowlin

In [19]:
%%time

def process_autor(x):
    r = x['AUTOR'].strip()
    s = re.findall('(.+)(, \d{4}-(\d{4}\.?)?)', r)
    if s:
        r = s[0][0]
    return r

autores_df['AUTOR'] = prestamos_df.apply(process_autor, axis=1)

CPU times: user 7min 25s, sys: 1.42 s, total: 7min 26s
Wall time: 7min 26s


## País ejemplar

In [36]:
total_countries = set(list(loc_marc_sr.values) + list(iso2_sr.values) + list(iso3_sr.values))

In [68]:
%%time

def process_pais(x):
    a = x['PAIS_EJEMPLAR'].strip()
    if a in loc_marc_sr:
        r = loc_marc_sr[a]
    elif '-' + a in loc_marc_sr:
        r = loc_marc_sr['-' + a]
    elif a in iso3_sr:
        r = iso3_sr[a]
    elif a in iso2_sr:
        r = iso2_sr[a]
    elif a in missing_codes_dict:
        r = missing_codes_dict[a]
    else:
        r = 'DESCONOCIDO'
    return r

autores_df['PAIS_EJEMPLAR'] = prestamos_df.apply(process_pais, axis=1)

CPU times: user 11min 58s, sys: 1.03 s, total: 11min 59s
Wall time: 11min 58s


In [70]:
autores_df['PAIS_EJEMPLAR'].value_counts()

Spain                                          1196618
Colombia                                        968719
Mexico                                          366627
Argentina                                       180773
DESCONOCIDO                                      86413
United States                                    49631
Venezuela                                        28224
England                                          27375
Chile                                            17621
New York (State)                                 15367
France                                           12416
California                                        9985
Peru                                              9473
Ecuador                                           9359
Brazil                                            8413
Cuba                                              7249
United Kingdom                                    6916
Italy                                             6338
Germany   

In [50]:
sorted(prestamos_df['PAIS_EJEMPLAR'].unique())

['',
 '-cn',
 '-cs',
 '-hk',
 '-uk',
 '-us',
 '99',
 '???',
 'AG',
 'CK',
 'SP',
 'a',
 'aa',
 'aag',
 'ab',
 'ae',
 'af',
 'ag',
 'aga',
 'agp',
 'ags',
 'alu',
 'an',
 'ao',
 'ap',
 'aq',
 'ar',
 'arc',
 'arg',
 'as',
 'at',
 'au',
 'aua',
 'aut',
 'azu',
 'b',
 'bb',
 'bbo',
 'bcc',
 'be',
 'bh',
 'bl',
 'bn',
 'bo',
 'br',
 'bu',
 'bus',
 'c',
 'ca',
 'cau',
 'cb',
 'cc',
 'ccc',
 'ccr',
 'ce',
 'cec',
 'cf',
 'ch',
 'ci',
 'cii',
 'ck',
 'ckd',
 'ckf',
 'ckk',
 'ckl',
 'cks',
 'cl',
 'cm',
 'cn',
 'cnu',
 'co',
 'col',
 'cou',
 'cpr',
 'cpy',
 'cr',
 'cs',
 'csp',
 'csu',
 'ctu',
 'cu',
 'cxk',
 'cz',
 'dcu',
 'de',
 'deu',
 'dk',
 'dn',
 'dp',
 'dr',
 'e',
 'ec',
 'ecu',
 'ed',
 'efr',
 'eg',
 'egx',
 'ek',
 'en',
 'ena',
 'eng',
 'enk',
 'ens',
 'ep',
 'es',
 'esp',
 'eu',
 'euk',
 'ev',
 'f',
 'fe',
 'fi',
 'fl',
 'flo',
 'flu',
 'fn',
 'fr',
 'fre',
 'fs',
 'fx',
 'g',
 'gau',
 'gb',
 'ge',
 'ger',
 'gk',
 'gp',
 'gr',
 'gs',
 'gt',
 'gu',
 'gw',
 'gx',
 'gxg',
 'gz',
 'he',
 

In [53]:
qqq=defaultdict(set)
for a in prestamos_df['PAIS_EJEMPLAR'].unique():
    if len(a) > 1:
        for c in total_countries:
            if c.lower().startswith(a.lower()):
                qqq[a].add(c)

In [55]:
len(qqq)

94

In [56]:
for k in qqq:
    if len(qqq[k]) > 1:
        print(k, '-', qqq[k])

tu - {'Tuvalu', 'Turkmenistan', 'Turkey', 'Turkmen S.S.R.', 'Tunisia', 'Turks and Caicos Islands'}
gr - {'Greenland', 'Grenada', 'Greece'}
ar - {'Argentina', 'Arizona', 'Armenia', 'Arkansas', 'Armenia (Republic)', 'Aruba', 'Armenian S.S.R.'}
ger - {'Germany (East)', 'Germany'}
cu - {'Curaçao', 'Cuba'}
jo - {'Johnston Atoll', 'Jordan'}
pe - {'Pennsylvania', 'Peru'}
bo - {'Bosnia and Herzegovina', 'Bouvet Island', 'Botswana', 'Bolivia'}
sy - {'Syrian Arab Republic (Syria)', 'Syria'}
fr - {'French Southern Territories', 'France', 'French Guiana', 'French Polynesia'}
ch - {'Christmas Island', 'China', 'China (Republic : 1949- )', 'Christmas Island (Indian Ocean)', 'Chile', 'Chad'}
por - {'Portugal', 'Portuguese Timor'}
ru - {'Russian Federation', 'Russian S.F.S.R.', 'Russia (Federation)'}
cz - {'Czechoslovakia', 'Czech Republic'}
br - {'British Columbia', 'Brazil', 'Brunei Darussalam', 'British Virgin Islands', 'British Indian Ocean Territory', 'Brunei'}
fi - {'Fiji', 'Finland'}
un - {'Uni

In [51]:
total_countries

{'Afghanistan',
 'Alabama',
 'Aland Islands',
 'Alaska',
 'Albania',
 'Alberta',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Arizona',
 'Arkansas',
 'Armenia',
 'Armenia (Republic)',
 'Armenian S.S.R.',
 'Aruba',
 'Ashmore and Cartier Islands',
 'Australia',
 'Australian Capital Territory',
 'Austria',
 'Azerbaijan',
 'Azerbaijan S.S.R.',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bermuda Islands',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Columbia',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Byelorussian S.S.R.',
 'Cabo Verde',
 'California',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Canal Zone',
 'Canton and Enderbury Islands',
 'Cape Verde',
 'Caribbean Netherlands',
 'Cayman Isl

In [254]:
sss=''
for x in autores_df['AUTOR'].unique():
    sss+=x+'\n'

In [255]:
with open('autores.txt', 'w') as f:
    f.write(sss)

In [169]:
prestamos_df['PAIS_EJEMPLAR'].nunique()

341

In [171]:
c

167

In [82]:
%%time

autores_dict['García Márquez, Gabriel, 1927-2014']

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


'García Márquez, Gabriel.'

In [100]:
%%time

process_autor('García Márquez, Gabriel, 1927-2014')

CPU times: user 17 µs, sys: 1 µs, total: 18 µs
Wall time: 21.5 µs


', 1927-2014'

In [ ]:
%%time

def process_autor(x):
    r = x['AUTOR'].strip().lower()
    r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return r

autores_df['AUTOR'] = ciudades_df.apply(process_field_departamento, axis=1)

In [9]:
prestamos_df['AUTOR'].value_counts()[:10]

                                            409373
García Márquez, Gabriel, 1927-2014           13207
Colombia.                                    11526
Hergé, 1907-1983.                            11510
Browne, Anthony, 1946-.                       8653
Foucault, Michel, 1926-1984.                  6849
Cortázar, Julio, 1914-1984.                   6478
Nietzsche, Friedrich Wilhelm, 1844-1900.      5876
Da Coll, Ivar, 1962-                          5350
Kasza, Keiko.                                 5336
Name: AUTOR, dtype: int64

In [17]:
prestamos_df['AUTOR'].value_counts()

                                                                409373
García Márquez, Gabriel, 1927-2014                               13207
Colombia.                                                        11526
Hergé, 1907-1983.                                                11510
Browne, Anthony, 1946-.                                           8653
Foucault, Michel, 1926-1984.                                      6849
Cortázar, Julio, 1914-1984.                                       6478
Nietzsche, Friedrich Wilhelm, 1844-1900.                          5876
Da Coll, Ivar, 1962-                                              5350
Kasza, Keiko.                                                     5336
Shakespeare, William, 1564-1616.                                  5012
Verne, Jules, 1828-1905.                                          4394
Instituto Colombiano de Normas Técnicas y Certificación.          4318
Dostoievski, Fedor Mijailovich, 1821-1881.                        4296
Rowlin

In [38]:
s='García Márquez, Gabriel, 1927-2014.'

In [57]:
s='Lindsay, Michael, 1909-'

In [53]:
s='Martínez Arango, María Elena.'

In [58]:
sss=re.findall('(, \d{4}-(\d{4}\.?)?)', s)

In [59]:
sss

[(', 1909-', '')]

In [55]:
sss[0][0]

IndexError: list index out of range

# Autores

In [2]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}